In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import  StandardScaler, MinMaxScaler
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
df = pd.read_csv('train_cdna_data.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/var/folders/cl/4zbw16hj6qb2lgkgb2840sb80000gn/T/ipykernel_28821/1430732877.py:1: DtypeWarning: Columns (7,12,13,15,16,17,18,19,29,37,45,46,47,56,58,66,69,71,74,228) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train_cdna_data.csv')


#### DATA CLEANING : 

In [3]:
df = df.dropna(axis=1, how='all')
missing_percentages = df.isnull().mean() * 100
df = df.loc[:, missing_percentages < 60]
df.shape
df = df.drop('v83', axis=1)
df = df.drop('v93', axis=1)
df = df.drop('v94', axis=1)
df['v37'] = df['v37'].fillna(df['v35'])
df['v37'] = df['v37'].fillna(df['v6'])
df = df.drop('v35', axis=1)
df = df.drop('v6', axis=1)
df = df.drop('v9', axis=1)
df = df.drop(columns=['v7'])
df = df.drop(columns=['v33'])
df = df.drop(columns=['v56'])
df['v37'] = df['v37'].fillna(df['v34'])
df = df.drop('v34', axis=1)
df = df.drop(columns=['v229','v230','v39','v40','v41'])
df = df.drop(columns=['v10','v31','v287','v288','v289','v290','v291','v292','v293','v294','v295','v296','v297','v298','v299','v300','v301','v302'])

In [4]:
df['batch_date'] = pd.to_datetime(df['batch_date'], errors='coerce')
def merge_customer_entries(df):
    df = df.dropna(subset=['batch_date'])
    df = df.sort_values(by=['CUSTOMER_CODE', 'batch_date'], ascending=[True, False])
    merged_data = df.groupby('CUSTOMER_CODE').apply(lambda group: group.ffill().bfill().iloc[0])
    return merged_data.reset_index(drop=True)

df = merge_customer_entries(df)

In [5]:
df['v27'] = df['v27'].replace({
    'UNMARRIED': 'Unmarried', 
    'S': 'Unmarried', 
    'Single': 'Unmarried', 
    'SINGLE': 'Unmarried',
    'WIDOW': 'Widow',
    'MARRIED': 'Married', 
    'M': 'Married'
})
value_counts = df['v27'].value_counts()

In [6]:
df['v29'] = df['v29'].replace({
    'SELFEMPLOYEDBUSINESS': 'Self Employed Business', 
    'SELF EMPLOYED BUSINESS': 'Self Employed Business', 
    'SELF EMPLOYED': 'Self Employed', 
    'SELFEMP': 'Self Employed',
    'House Wife': 'Homemaker',
    'HOUSE WIFE': 'Homemaker',
    'OTHERSHOUSEWIFE': 'Homemaker',
    'HOMEMAKER':'homemaker',
    'Salarid-Public': 'Salaried-Public',
    'Salarid-Govt': 'Salaried-Govt',
    'Salaried - Govt' : 'Salaried-Govt',
    'RETIRED':'Retired',
    'FARMER': 'Farmer',
    'PENSIONER': 'Retired',
    'Retd. Pensioner': 'Retired',
    'TVS' : 'Employee',
    'EMPLOYEE': 'Employee',
    'Company labour':'Employee',
    'GROUP COMPANY EMPLOYEE' : 'Employee',
    'OTHERSSTUDENT':'Student',
    'STUDENT':'Student',
    'Salaried-Others':'Salaried',
    'SALARIED':'Salaried',
    'OTHERS':'OTHER',
    'OTHERSNOTCATEGORIZED':'OTHER',
    'INDIVIDUAL OTHERS':'OTHER',
    'INDIVIDUAL SELF EMPLOYED':'Self Employed',
    'INDIVIDUAL SALARIED':'Salaried',
    'Selp employed profesional': 'Self employed professional',
    'SELFEMPLOYEDPROFESSIONAL': 'Self employed professional',
    'AlliedAgri-Dairy':'Allied Agri-Dairy',
    'PROFESSIONAL':'Professional',
    'Agri-Marginal Farmer up to 2.5 Acre':'Agri-Marginal Farmer (up to 2.5 Acre)',
    'Self employed Non Professional': 'SELF EMPLOYED - NON PROFESSIONAL',
    'Labour-OtherLabour':'Labour',
    'Labour-Other  Labour':'Labour',
    'Agri-Small Farmer 2.5-5 Acre':'Agri-Small Farmer (2.5-5 Acre)',
    'Service-Tailor':'Tailor',
    'Labour-AgriLabour':'AgriLabour',
    'EMP' : 'Employee',
})

df['v54'] = df['v54'].replace({
    'MALE': 'Male' , 
    'M': 'Male', 
    'FEMALE': 'Female', 
    'F': 'Female',
    'THIRD GENDER': 'OTHERS',
    'U': 'OTHERS', 
    'O': 'OTHERS',
    'C':'OTHERS',
})

df['v55'] = df['v55'].str.split('.').str[0].str[1:]
df['v55'] = df['v55'].replace({
    'YMAIL': 'GMAIL', 
    'GAMIL': 'GMAIL', 
    'GMAI': 'GMAIL', 
    'GMAL': 'GMAIL',
    'GOOGLEMAIL': 'GMAIL',
    'REDIFF': 'REDIFFMAIL',
    'HMAIL': 'GMAIL',
    'GMIL': 'GMAIL',
    '123GMAIL': 'GMAIL',
    'GMIL' : 'GMAIL',
    'GEMAL':'GMAIL',
    'GMSIL': 'GMAIL',
    'GMIAL': 'GMAIL',
    'GMAAIL': 'GMAIL',
    'GIMAL' : 'GMAIL',
    'GMEIL': 'GMAIL',
    'REDIFMAIL':'REDIFFMAIL',
    'GMAIP' : 'GMAIL',
    'GNAIL':'GMAIL',
    'YOHOO':'YAHOO',
    'GMAOL':'GMAIL',
    '32GMAIL':'GMAIL',
    'GMIEL':'GMAIL',
    'GMAIOL':'GMAIL',
    'YHOO':'YAHOO',
    'GMAIO':'GMAIL',
    'GMQIL':'GMAIL',
    'GMAIK': 'GMAIL',
})

df = df.drop('v11', axis=1)
df[['v5']] = df[['v5']].replace('ZZ', np.nan)
df[['v30']] = df[['v30']].replace(False, np.nan)
df[['v30']] = df[['v30']].replace('ZZ', np.nan)
df[['v30']] = df[['v30']].replace('false', np.nan)
boolean_columns = ['v71', 'v73']  
df[boolean_columns] = df[boolean_columns].fillna(True)
df['v30'] = pd.to_numeric(df['v30'], errors='coerce')
df['v65'] = df['v65'].fillna(0.0)

In [7]:
def replace_rare_values(df, column, threshold=20):
    value_counts = df[column].value_counts()
    rare_values = value_counts[value_counts < threshold].index
    df[column] = df[column].replace(rare_values, 'Others')
    return df

df = replace_rare_values(df, 'v29', threshold=20)
df = replace_rare_values(df, 'v55', threshold=21)

#### Imputation : 

In [8]:
object_columns = df.select_dtypes(include=['object']).columns
for col in object_columns:
    mode = df[col].mode()[0]  
    df[col].fillna(mode, inplace=True)
numeric_columns = df.select_dtypes(include=['number']).columns
df[numeric_columns] = df[numeric_columns].interpolate(method='linear', axis=0,limit_direction='both')

#### Feature Scaling :

In [9]:
int_columns = df.select_dtypes(include=['int64', 'int32']).columns
scaler = MinMaxScaler()
df[int_columns] = scaler.fit_transform(df[int_columns])

float_columns = df.select_dtypes(include=['float64']).columns
scaler = StandardScaler()
df[float_columns] = scaler.fit_transform(df[float_columns])

#### Feature Encoding :

In [10]:
df['v60'] = df['v60'].apply(lambda x: 1 if x == 'N' else 0)
df['v60'] = df['v60'].astype(int)
boolean_columns = df.select_dtypes(include=['bool']).columns
for col in boolean_columns:
    df[col] = df[col].astype(int)

In [11]:
exclude_columns = ['CUSTOMER_CODE']
object_columns = [col for col in df.select_dtypes(include=['object']).columns if col not in exclude_columns]
label_encoders = {}
for col in object_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  

In [12]:
df.dtypes

CUSTOMER_CODE            object
v2                        int64
v5                        int64
v15                       int64
v24                     float64
v27                       int64
v29                       int64
v30                     float64
v32                     float64
v36                       int64
v37                       int64
v42                       int64
v43                       int64
v44                     float64
v45                       int64
v49                       int64
v50                       int64
v51                       int64
v52                       int64
v53                       int64
v54                       int64
v55                       int64
v60                       int64
v63                       int64
v65                     float64
v66                       int64
v68                       int64
v69                       int64
v71                       int64
v73                       int64
v74                       int64
v77     

In [13]:
df = df.drop(columns=['batch_date'])

In [14]:
df.head()

,CUSTOMER_CODE,v2,v5,v15,v24,v27,v29,v30,v32,v36,v37,v42,v43,v44,v45,v49,v50,v51,v52,v53,v54,v55,v60,v63,v65,v66,v68,v69,v71,v73,v74,v77,v79,v80,v81,v82,v84,v86,v87,v88,v89,v90,v91,v92,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v128,v129,v130,v203,v204,v205,v206,v207,v208,v209,v224,v225,v226,v227,v228,v238,v239,v240,v241,v243,v244,v252,v253,v254,v255,v257,v258,v259,v260,v261,v262,v263,v264,v265,v266,v267,v268,v269,v270,v271,v272,v273,v274,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286
0,00006eb623a7a335fb6fa7334939cca08c75e80e44e404...,8,23,30,-0.262437,0,12,1.418758,-0.691350,2,148,62,0,-0.625053,1,0,1,1,0,1,1,1,1,2,-0.824299,1,1,1,1,0,0,-0.305365,-0.14122,0.278696,1,-0.590127,-0.073599,-0.165676,-0.526604,1.124648,0.032718,0.021503,-0.080927,-0.148788,-0.401612,-0.238922,-0.091035,-0.141264,0,-0.19407,1,4,1,-0.952885,-0.755498,1.619092,-0.684390,-0.799676,-0.493873,-0.224518,-0.063051,-0.137233,-0.223058,-0.048590,0.005153,-0.136979,-0.296417,0.078413,-0.105369,-0.367708,-0.048590,-0.144380,0.150994,0.004879,-0.122452,0.000309,-0.005196,-0.041246,0.282872,0.129219,-0.153399,0.509873,0.543757,-0.511840,-0.498629,-0.438464,-0.453973,-0.098491,-0.110599,-0.119719,-0.124731,-0.517695,-0.505813,-0.446858,-0.463175,1,1,1,1,1,1,1,1,2,2,1,2,1,2,1,2
1,000095ace79862eb5f4481723b8757b8637ba7ab5caeea...,4,23,30,-0.262437,2,17,-0.692285,-0.500484,2,517,62,3,-0.625053,1,0,1,0,0,0,1,1,1,1,1.213152,1,1,1,0,0,0,-0.305365,-0.14122,-1.068573,1,1.694551,-0.073599,-0.165676,-0.526604,-0.889167,0.032718,0.021503,-0.080927,-0.148788,-0.401612,-0.238922,-0.091035,-0.141264,0,-0.19407,0,5,1,1.049445,-0.755498,-0.617630,-0.684390,-0.799676,-0.493873,-0.224518,-0.063051,-0.137233,-0.223058,-0.048590,0.005153,-0.136979,-0.296417,0.078413,-0.105369,-0.367708,-0.048590,0.171931,0.611005,0.413344,-0.181770,0.003225,-0.002081,0.273226,0.743817,0.537839,-0.200669,0.510017,0.543895,-0.395649,-0.363716,-0.403930,-0.314503,-0.098491,-0.110599,-0.119719,-0.124731,-0.403006,-0.372527,-0.412746,-0.325543,1,1,1,1,1,1,1,1,1,2,1,2,1,2,1,2
2,0000b7d57830b87a5049eca606772266fc83c60e690fb4...,9,23,30,-0.262437,0,16,1.418758,-0.500484,2,314,62,0,-0.625053,1,0,1,1,1,0,1,1,1,2,-0.824299,1,1,1,1,0,0,-0.305365,-0.14122,0.817603,1,-0.590127,-0.073599,-0.165676,-0.526604,1.124648,0.032718,0.021503,-0.080927,-0.148788,-0.401612,-0.238922,-0.091035,-0.141264,0,-0.19407,1,5,1,-0.952885,-0.755498,-0.617630,-0.560857,-0.640601,-0.493873,-0.497812,-0.350580,-0.428172,-0.223058,-0.044750,0.005709,-0.136532,-0.560109,-0.215438,-0.406010,-0.367708,-0.044750,0.488241,1.071015,0.821809,-0.241088,0.006140,0.001035,0.587698,1.204762,0.946458,-0.247939,0.510162,0.544034,-0.279459,-0.329988,-0.311838,-0.286609,-0.098491,-0.110599,-0.119719,-0.124731,-0.288318,-0.339206,-0.321781,-0.298016,1,1,1,1,1,1,1,1,1,2,1,2,1,2,1,2
3,0000b844339117db33d8dcc6ca4e97a8c989a918c8cf01...,4,23,30,-0.262437,2,12,-0.810883,-0.500484,2,517,62,3,-0.625053,1,0,1,0,0,0,0,1,1,2,1.213152,1,0,1,0,0,0,-0.305365,-0.14122,-1.338027,1,-0.590127,-0.073599,-0.165676,-0.526604,1.124648,0.032718,0.021503,-0.080927,-0.148788,-0.401612,-0.238922,-0.091035,-0.141264,1,-0.19407,1,4,1,1.049445,-0.755498,-0.617630,-0.437324,-0.481525,-0.493873,-0.771105,-0.638108,-0.719112,-0.223058,-0.040910,0.006266,-0.136085,-0.823800,-0.509288,-0.706651,-0.367708,-0.040910,0.694928,1.305226,1.055587,-0.241088,-0.239744,-0.261710,0.793183,1.439449,1.180324,-0.247939,0.497999,0.532319,-0.163268,-0.296260,-0.219746,-0.258715,-0.098491,-0.110599,-0.119719,-0.124731,-0.173629,-0.305885,-0.230816,-0.270490,1,1,0,1,1,1,1,1,1,2,1,2,1,2,1,2
4,0000d719d7706b7164235a7c7dc4a1954799eff90fb622...,9,23,30,-0.262437,0,14,-0.810883,-0.500484,2,517,62,0,0.843525,1,0,1,0,0,0,1,1,1,2,1.213152,1,1,1,0,0,0,-0.305365,-0.14122,0.817603,1,-0.590127,-0.073599,-0.165676,-0.526604,-0.889167,0.032718,0.021503,-0.080927,-0.148788,-0.401612,-0.238922,-0.091035,-0.141264,0,-0.19407,0,4,1,1.049445,-0.755498,-0.617630,-0.684390,-0.799676,-0.493873,-0.6

In [15]:
df.to_csv('CDNA.csv', index=False)